In [1]:
import re
import nltk
import string
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
def pre_processing(sentence):
    return remove_stopwords(tokenize_sentence(remove_punctuation(sentence)))

# Remove punctuation from a list of words
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

# Remove stopwords from a list of words
def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

# Tokenize the input sentence and also lemmatize its words
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif (tag[1][:2] == "VB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif (tag[1][:2] == "RB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif (tag[1][:2] == "JJ"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

def get_signature(sense):
    signature = []
    for word in tokenize_sentence(sense.definition()):  # definition tokenization
        signature.append(word)
    for example in sense.examples():  # example tokenization
        for word in tokenize_sentence(example):
            # Merge definition and examples
            signature.append(word)
    return signature

Read corpus file. It return a list of list of documents and words for each document.

In [3]:
def read_corpus(txt_file):
    
    with open(txt_file, encoding='utf-8') as file:
        # for each doc create list of pre-processed words in that doc (list of lists)
        documents_words = []

        for line in file:
            if "<doc" in line:  # tag for new doc
                document_words = []  # list of words that will be part of the document
                while True:
                    next_line = file.readline()  # read next line

                    # remove unuseful tags
                    next_line_proc = next_line.replace("<p> ", "").replace("</p>\n", "").replace("/p", "")

                    if "</doc>" in next_line:
                        break
                    # pre-processing steps
                    sentence_words = pre_processing(next_line_proc)
                    document_words.extend(sentence_words)
                documents_words.append(document_words)
        file.close()

    print("Documents number: ", len(documents_words))
    
    return documents_words

Topic modelling using gensim

In [4]:
from gensim import corpora, models

topic_num = 10
topic_words_num = 5

def topic_modelling(documents_words):
    
    
    # Create a dict with integer keys for all words
    dictionary_LDA = corpora.Dictionary(documents_words)

    # delete all terms that do NOT appear in at least 3 documents.
    #delete all terms that appear in more than 60% of documents (see filter_extremes official doc).
    dictionary_LDA.filter_extremes(no_below=3, no_above=0.6)

    # Converts each document into a list of BoW (list of (id_term, term_frequency) for each term in doc)
    corpus_idbow_freq = [dictionary_LDA.doc2bow(document_words) for document_words in documents_words]
    
    # https://radimrehurek.com/gensim/models/ldamodel.html
    lda_model = models.LdaModel(corpus_idbow_freq, num_topics=topic_num, \
                                id2word=dictionary_LDA, \
                                passes=3, alpha=[0.01] * topic_num, \
                                eta=[0.01] * len(dictionary_LDA.keys()))
    
    return lda_model, corpus_idbow_freq


Show topics.

In [5]:
documents_words = read_corpus("travelling.txt")

model, corpus_idbow_freq = topic_modelling(documents_words)

topics = {'Topic ' + str(i): [(token, round(score, 3)) for token, score in model.show_topic(i, topn=topic_words_num)] for i in range(0, model.num_topics)}
for key, value in topics.items():
    print(key,":", topics[key])

Documents number:  100
Topic 0 : [('clause', 0.045), ('example', 0.02), ('third', 0.02), ('perfect', 0.016), ('conditionals', 0.014)]
Topic 1 : [('travel', 0.017), ('money', 0.014), ('lot', 0.011), ('holiday', 0.011), ('dream', 0.01)]
Topic 2 : [('clause', 0.026), ('situation', 0.015), ('condition', 0.015), ('result', 0.015), ('happen', 0.015)]
Topic 3 : [('students', 0.065), ('exam', 0.035), ('book', 0.026), ('sb', 0.026), ('speak', 0.025)]
Topic 4 : [('word', 0.023), ('clause', 0.021), ('happen', 0.014), ('example', 0.013), ('noun', 0.013)]
Topic 5 : [('level', 0.02), ('teach', 0.016), ('video', 0.015), ('love', 0.015), ('esl', 0.015)]
Topic 6 : [('hotel', 0.038), ('book', 0.035), ('holiday', 0.025), ('beach', 0.023), ('travel', 0.016)]
Topic 7 : [('lesson', 0.038), ('student', 0.012), ('grammar', 0.01), ('learn', 0.01), ('language', 0.009)]
Topic 8 : [('clause', 0.015), ('condition', 0.013), ('happen', 0.012), ('result', 0.012), ('book', 0.01)]
Topic 9 : [('travel', 0.031), ('articl

Show topics for each document.

In [6]:
print ("Documents topic list")
for i in range (0, len(corpus_idbow_freq)):
    print ("Doc", i, ":", model[corpus_idbow_freq[i]])

Documents topic list
Doc 0 : [(1, 0.8516115), (9, 0.14705722)]
Doc 1 : [(1, 0.1281324), (3, 0.2067905), (6, 0.6648527)]
Doc 2 : [(1, 0.43143946), (7, 0.5667039)]
Doc 3 : [(0, 0.7186929), (3, 0.101707526), (7, 0.05541384), (9, 0.12384491)]
Doc 4 : [(7, 0.99982)]
Doc 5 : [(3, 0.08431996), (4, 0.04471661), (7, 0.46974307), (9, 0.40117958)]
Doc 6 : [(7, 0.99901193)]
Doc 7 : [(0, 0.7027754), (2, 0.11124558), (7, 0.18546453)]
Doc 8 : [(1, 0.99991614)]
Doc 9 : [(1, 0.999696)]
Doc 10 : [(2, 0.73135585), (4, 0.26853248)]
Doc 11 : [(4, 0.99972486)]
Doc 12 : [(8, 0.9991354)]
Doc 13 : [(5, 0.09053723), (7, 0.9091687)]
Doc 14 : [(5, 0.7187402), (7, 0.25519317), (9, 0.02567997)]
Doc 15 : [(1, 0.9998529)]
Doc 16 : [(2, 0.15852156), (4, 0.3549142), (8, 0.2116178), (9, 0.27447054)]
Doc 17 : [(1, 0.9090508), (4, 0.0344841), (5, 0.016202021), (9, 0.040096387)]
Doc 18 : [(2, 0.84581715), (4, 0.15388443)]
Doc 19 : [(0, 0.2047706), (2, 0.20065314), (4, 0.55849), (7, 0.035849158)]
Doc 20 : [(2, 0.26951432), 